In [3]:

import sys
sys.path.append('C:/Users/LUCP12511/Documents/PhD Kato/ISALA/Repos/BIU-master') # Modify this to where you have downloaded the BIU library
#import biu
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
plt.rcParams['svg.fonttype'] = 'none'

import seaborn as sns
import scipy

# install rdmpy with pip install git+https://github.com/thiesgehrmann/rdmpy
#from rdmpy import RDM
#RDM.meta(source="repos/LunaQuestionnaire/clean.ipynb")

# Cleaned questions overview table

|Question Number|Question identifier|Clean question identifier|
|---------------|-------------------|-------------------------|
|   |   | Reproductive.Cycle.Phase|
|Q1.Q52.2.|Heeft dit te maken met één van de onderstaande mogelijkheden? - Selected Choice|Reproductive.PeriMenopause|
|Q1.Q4.|Mijn partner(s) is/zijn...|Lifestyle.Partner.Is|
| |uitgedragen|Reproductive.Pregnancy|
|Q1.Q21.|Rook je? Geef ook op hoeveel sigaretten je per dag rook(te) - Selected Choice|Lifestyle.Drugs.Current_smoker|
|Q1.Q22.|Gebruik je drugs (exclusief alcohol)? Gelieve ook aan te geven welke. Deze informatie wordt pseudoniem verwerkt. Wij hebben het grootste respect voor jou en voor je privacy. - Selected Choice|Lifestyle.Drugs.Current_druguser|
|Q1.Q49.|Heb je ooit actief geprobeerd om zwanger te worden?|Reproductive.Pregnancy.Attempted|
|Q2.Q24.|Geef je op dit moment borstvoeding?|Reproductive.Breastfeeding|
|Q1.Q29_4.|Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Vlees|Nutrition.EatsMeat|
|Q1.Q29_6.|Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Vis|Nutrition.EatsFish|
|Q1.Q29_5.|Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Dierlijke producten (eieren, kaas, gelatine,...)|Nutrition.EatsAnimals|
| | |Nutrition.Vgetarian|
| | |Nutrition.Pescetarian|
| | |Nutrition.Vegan|
|Q1.Q15.|Hoe zou je het gebied waar je nu woont omschrijven?|Environment.Urban|
|Q1.Q15.|Hoe zou je het gebied waar je nu woont omschrijven?|Environment.Coutryside|
|Q1.Q9.|In welk(e) land(en) heb je gewoond tot en met je derde levensjaar? Indien dit enkel België is, mag je het ook opgeven.|Environment.Belgium.First3Years|
|Q1.Q47.|Heb je de afgelopen drie maanden één of meerdere seksuele\npartner(s) gehad? - Selected Choice|Lifestyle.Partner.OneVsSingle|
|Q1.Q47.|Heb je de afgelopen drie maanden één of meerdere seksuele\npartner(s) gehad? - Selected Choice|Lifestyle.Partner.OneVsMany|
|Q2.Q7.|Hoe groot is je aandeel aan het gezinsinkomen?|Environment.Economic.HouseholdContribution|
|Q2.Q4.|Kan je maandelijks rondkomen met de som van totaal beschikbaar gezinsinkomen (alles inbegrepen*)?|Environment.Economic.FinancialDifficulties|
|Q1.Q50.|Hoeveel keer was je al zwanger?|Reproductive.Pregnancy.n|
|Q1.Q1.|Geslacht - Selected Choice|General.Gender|
|Q1.Q2_1.|Enkele algemene vragen over jou: - Leeftijd (jaar)|General.Age|
|Q1.Q2_2.|Enkele algemene vragen over jou: - Lengte (cm)|General.Height|
|Q1.Q2_3.|Enkele algemene vragen over jou: - Gewicht (kg)|General.Weight|
| | |General.BMI



# Functions

In [9]:
def freq(lst, key=lambda x: x):
    """
    Count the number of occurances in a list.
    Input:
        lst: an iterable list
        key: a callable function that is called upon the value, the return value of which is counted
    
    Output: Dictionary with count of each item
    """
    F = {}
    for item in lst:
        if pd.isna(item):
            item = None
        #fi
        F[key(item)] = F.get(key(item), 0) + 1
    #efor
    return F


def qerf(lst, key=lambda x: x):
    """
    Return the frequency of objects in a list, indexed by their frequency, rather than by the item itself.
    parameters:
    -----------
    lst: list[obj]
        List of hashable objects
    key: callable(obj)
        A function which will be called upon the objects in the list
        
    example:
    qerf([1,2,3,3,4,4,4])
     { 1: [1,2],
       2: [3],
       3: [4]
       }
    """
    f = freq(lst, key)
    q = { }
    for (i,c) in f.items():
        q[c] = q.get(c,[]) + [i]
    #efor
    return q

# Load the Isala Flow 1 questionnaire data

In [10]:
#with RDM('../data/processed/merged_questionnaires.pkl', 'r', nouser=True) as ifd:
#    Qraw = pd.read_pickle(ifd)
#ewith
Qraw = pd.read_pickle("../data/processed/merged_questionnaires.rdm_thies_2024-08-09T11-49-27-140516.pkl")
    
#Qraw = pd.read_excel('../data/processed/merged_from_sharepoint_flow1_all_questionnaires.20231031.xlsx')

## Search for questions
Type in words that are part of a question in order to get back the names of matching columns

In [11]:
from ipywidgets import interact, interactive, fixed, interact_manual
## Interactive questionnaire search
def search_questionnaire(query):
    if len(query) < 2:
        return None
    #fi
    return Qraw[[ x for x in Qraw.columns if query.lower() in x.lower() ]].transpose()
#edef

_ = interact(search_questionnaire, query='')

interactive(children=(Text(value='', description='query'), Output()), _dom_classes=('widget-interact',))

# Prepare the cleaned dataframe

In [12]:
QC = Qraw[['Q0.Deelnemersnummer']].rename(columns={'Q0.Deelnemersnummer':'IsalaID'}).copy()

# Clean questions

## Menstrual Cycle 

In [13]:
def determine_phases(row):
    """
    returns
     string
    """
    cycle_disrupted = row['Q1.Q52.3.[Hoeveel dagen zijn er gemiddeld tussen de eerste dag van je menstruatie\nen de eerste dag van je volgende? Je hoeft enkel het aantal dagen in te vullen indien je geen hormonale anticonceptie gebruikt die je cyclus kan beïnvloeden. - Selected Choice]'] == "Ik gebruik hormonale anticonceptie of een andere hormonale behandeling die mijn menstruele cyclus kan beïnvloeden"
    cycle_days     = row['Q1.Q52.3_4_TEXT.[Hoeveel dagen zijn er gemiddeld tussen de eerste dag van je menstruatie\nen de eerste dag van je volgende? Je hoeft enkel het aantal dagen in te vullen indien je geen hormonale anticonceptie gebruikt die je cyclus kan beïnvloeden. - Ik gebruik geen hormonale anticonceptie of een andere hormonale behandeling die mijn menstruele cyclus kan beïnvloeden - Tekst]']
    current_day    = row['A.day_of_cycle']
    
    if cycle_disrupted:
        return 'not_applicable'
    #fi
    
    if isinstance(cycle_days, str):
        return "unknown"
    #fi
    
    if pd.isna(cycle_days):
        cycle_days = 28
    #fi
    if pd.isna(current_day):
        return 'unknown'
    #fi
    
    if (cycle_days < 21) or (cycle_days > 35):
        return 'unknown'
    #fi
    
    if cycle_days <= 24:
        ovulation = cycle_days - 12
    elif cycle_days <= 31:
        ovulation = cycle_days - 14
    else:
        ovulation = cycle_days - 16
    #fi
    
    if current_day <= (ovulation-6):
        return 'Follicular'
    elif current_day <= (ovulation+2):
        return 'Ovulation'
    else:
        return 'Luteal'
    #fi
    
    return 'unknown'
#edef

phase = Qraw.apply(determine_phases, axis=1)



col_values = ['Luteal', 'Ovulation', 'Follicular']
for cv in col_values:
    QC['Reproductive.Cycle.Phase.%s' % cv] = phase.apply(lambda x: True if x == cv else None if x not in col_values else False)
#efor

QC['Reproductive.PeriMenopause'] = Qraw['Q1.Q52.2.[Heeft dit te maken met één van de onderstaande mogelijkheden? - Selected Choice]'].apply(lambda x: str(x).lower() in ['c','d'])


## Partner gender

In [14]:
col = 'Q1.Q4.[Mijn partner(s) is/zijn...]'
freq(Qraw[col])
col_values = {'Man' : 'Man', 'Vrouw' : 'Woman'}
for cv in col_values:
    QC['Lifestyle.Partner.Is%s' % col_values[cv]] = Qraw[col].apply(lambda x: True if x == cv else None if x not in col_values else False)
    print('Lifestyle.Partner.Is%s' % col_values[cv])
#efor

Lifestyle.Partner.IsMan
Lifestyle.Partner.IsWoman


## Miscarriage, abortions

In [15]:
_rel = Qraw[[c for c in Qraw if 'uitgedragen' in c]]

for rtype, rvalues in {'Miscarriage': ['Nee afgebroken, miskraam met medicatie', 'Nee afgebroken, spontane miskraam', 'Nee afgebroken, curettage miskraam'],
                      'PretermBirth': ['Nee, vroeggeboorte'],
                      'Abortion': ['Nee afgebroken, abortus']}.items():
    QC['Reproductive.Pregnancy.%s' % rtype] = _rel.applymap(lambda x: x in rvalues).any(axis=1)
#efor

C:\Users\LUCP12511\AppData\Local\Temp\ipykernel_39332\1108683355.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  QC['Reproductive.Pregnancy.%s' % rtype] = _rel.applymap(lambda x: x in rvalues).any(axis=1)
C:\Users\LUCP12511\AppData\Local\Temp\ipykernel_39332\1108683355.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  QC['Reproductive.Pregnancy.%s' % rtype] = _rel.applymap(lambda x: x in rvalues).any(axis=1)
C:\Users\LUCP12511\AppData\Local\Temp\ipykernel_39332\1108683355.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  QC['Reproductive.Pregnancy.%s' % rtype] = _rel.applymap(lambda x: x in rvalues).any(axis=1)


## Some previous questions

In [16]:

QC['Lifestyle.Drugs.Current_smoker'] = Qraw['Q1.Q21.[Rook je? Geef ook op hoeveel sigaretten je per dag rook(te) - Selected Choice]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

QC['Lifestyle.Drugs.Current_druguser'] = Qraw['Q1.Q22.[Gebruik je drugs (exclusief alcohol)? Gelieve ook aan te geven welke. Deze informatie wordt pseudoniem verwerkt. Wij hebben het grootste respect voor jou en voor je privacy. - Selected Choice]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

QC['Reproductive.Pregnancy.Attempted'] = Qraw['Q1.Q49.[Heb je ooit actief geprobeerd om zwanger te worden?]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

QC['Reproductive.Breastfeeding'] = Qraw['Q2.Q24.[Geef je op dit moment borstvoeding?]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

In [17]:
QC['Nutrition.EatsMeat'] = Qraw['Q1.Q29_4.[Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Vlees]'].apply(
    lambda x: None if pd.isna(x) else x not in ['Zelden', 'Nooit'])

QC['Nutrition.EatsFish'] = Qraw['Q1.Q29_6.[Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Vis]'].apply(
    lambda x: None if pd.isna(x) else x in ['Zelden', 'Nooit'])

QC['Nutrition.EatsAnimals'] = Qraw['Q1.Q29_5.[Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Dierlijke producten (eieren, kaas, gelatine,...)]'].apply(
    lambda x: None if pd.isna(x) else x in ['Zelden', 'Nooit'])

QC['Nutrition.Vgetarian'] = QC.apply(
    lambda x: None if any(pd.isna(x[['Nutrition.EatsMeat','Nutrition.EatsFish']])) else not(x['Nutrition.EatsFish'] or x['Nutrition.EatsMeat']), axis=1)

QC['Nutrition.Pescetarian'] = QC.apply(
    lambda x: None if any(pd.isna(x[['Nutrition.EatsMeat','Nutrition.EatsFish']])) else x['Nutrition.EatsFish'] and not(x['Nutrition.EatsMeat']), axis=1)

QC['Nutrition.Vegan'] = QC.apply(
    lambda x: None if any(pd.isna(x[['Nutrition.EatsMeat','Nutrition.EatsFish', 'Nutrition.EatsAnimals']])) else not(x['Nutrition.EatsFish'] or x['Nutrition.EatsMeat'] or x['Nutrition.EatsAnimals']), axis=1)


QC['Environment.Urban'] = Qraw['Q1.Q15.[Hoe zou je het gebied waar je nu woont omschrijven?]'].apply(
    lambda x: None if pd.isna(x) else x in ['Stadskern', 'Dorpskern', 'Residentiële woonwijk', 'Drukke baan', 'Industriezone'])

QC['Environment.Coutryside'] = Qraw['Q1.Q15.[Hoe zou je het gebied waar je nu woont omschrijven?]'].apply(
    lambda x: None if pd.isna(x) else x in ['Groene zone/ recreatiegebied/...', 'Landelijk gebied'])



In [18]:
def is_belgium(v):
    if pd.isna(v):
        return None
    #fi
    v = v.lower()
    if 'belg' in v:
        return True
    elif v in ['be', 'antwerpen', 'neen', '0', 'geen', 'b', 'beglie' ]:
        return True
    else:
        return False
    #fi
#edef

QC['Environment.Belgium.First3Years'] = Qraw['Q1.Q9.[In welk(e) land(en) heb je gewoond tot en met je derde levensjaar? Indien dit enkel België is, mag je het ook opgeven.]'].apply(
    lambda x: is_belgium(x))

QC['Lifestyle.Partner.OneVsSingle'] = Qraw['Q1.Q47.[Heb je de afgelopen drie maanden één of meerdere seksuele\npartner(s) gehad? - Selected Choice]'].apply(
    lambda x: None if (pd.isna(x) or x in ['Ja, meerdere vaste partners: hoeveel?', 'Ja, wisselende partners: hoeveel?']) else x == 'Ja, één vaste partner')
QC['Lifestyle.Partner.OneVsMany'] = Qraw['Q1.Q47.[Heb je de afgelopen drie maanden één of meerdere seksuele\npartner(s) gehad? - Selected Choice]'].apply(
    lambda x: None if (pd.isna(x) or x in ['Nee']) else x != 'Ja, één vaste partner')

QC['Environment.Economic.HouseholdContribution'] = Qraw['Q2.Q7.[Hoe groot is je aandeel aan het gezinsinkomen?]'].apply(
    lambda x: None if pd.isna(x) else {'Ik heb het grootste aandeel ': 3/4,
 'Ik draag als enige bij ': 4/4,
 'We dragen evenveel bij  ': 2/4,
 'Ik draag het minste bij  ': 1/4,
 'Ik draag niet bij  ': 0/4}.get(x, None))

QC['Environment.Economic.FinancialDifficulties'] = Qraw['Q2.Q4.[Kan je maandelijks rondkomen met de som van totaal beschikbaar gezinsinkomen (alles inbegrepen*)? \n\n\n\n* De leefomstandigheden van een huishouden hangen natuurlijk in belangrijke mate af van het inkomen. Het gaat hier over het totale beschikbare inkomen van de verschillende leden van het huishouden samen. Meerdere leden kunnen dus bijdragen. Het totale beschikbare inkomen van een huishouden bestaat uit, (1) netto-lonen, wedden en nettobedrijfsinkomens voor zelfstandigen, (2) sociale uitkeringen en kinderbijslagen, (3) bijkomende inkomens zoals huuropbrengsten, intresten e.a. De som van al deze inkomens voor alle personen uit je huishouden is het totale beschikbare inkomen van je huishouden.]'].apply(
    lambda x: None if pd.isna(x) else (x in ['Eerder moeilijk', 'Moeilijk', 'Zeer moeilijk']))

## General

### Gender

In [36]:
#Sex
def clean_gender(val):
    if pd.isna(val):
        return None
    elif 'Vrouw' in val:
        return 'Vrouw'
    elif 'Anders' in val:
        return 'Other'
    else:
        return None
    #fi
#edef
QC['General.Gender'] = Qraw["Q1.Q1.[Geslacht - Selected Choice]"].apply(clean_gender)

### Age

In [42]:
#Age
def clean_age(x): 
    if pd.isna(x):
        return None
    elif str(x).isnumeric():
        if x <= 200:
            return int(x)
        elif x > 200:
            return int(2020 - x)
        #fi
    else:
        return None
    #fi
#edef
QC['General.Age'] = Qraw["Q1.Q2_1.[Enkele algemene vragen over jou: - Leeftijd (jaar)]"].apply(clean_age)

### Height

In [51]:
#Height
def clean_height(x): 
    if pd.isna(x):
        return None
    elif str(x).isnumeric():
        if x > 3:
            return int(x)
        elif x <= 3:
            return int(x * 100)
        #fi
    else:
        return None
    #fi
#edef
QC['General.Height'] = Qraw["Q1.Q2_2.[Enkele algemene vragen over jou: - Lengte (cm)]"].apply(clean_height)

### Weight

In [59]:
#Weight

def clean_weight(val):
    if pd.isna(val):
        return None
    elif str(val).isnumeric():
        return float(val)
    else:
        return None
    #fi
#edef

QC['General.Weight'] = Qraw['Q1.Q2_3.[Enkele algemene vragen over jou: - Gewicht (kg)]'].apply(clean_weight)

### Education

In [195]:
map_main = {            #dictionnary for question Q1.Q18
'Postuniversitair: doctor': 'PhD',
 'Hogeschool: bachelor': 'College',
 'Universiteit: bachelor/master (kandidaat/licenciaat)': 'University',
 'Secundair onderwijs: KSO': 'Secondary education',
 'Secundair onderwijs: ASO': 'Secondary education',
 'Secundair onderwijs: BSO': 'Secondary education',
 'Secundair onderwijs: TSO': 'Secondary education',
 'Lagere school': 'Primary education',
 'Geen': 'No',
 '': None  
}

map_other = {  #If still studying: refers to highest completed degree
 'Hogeschool master': 'College',
 'Hogeschool - postgraduaat': 'College',
 'Master beeldende kunsten aan een hogeschool': 'College',
 'Hogeschool postgraduaat na bachelor': 'College',
 'professionele bachelor dans': 'College',
 'Hogeschool: master': 'College',
 'Dit is niet relevant ': None,
 'Hogeschool: bachelor -&gt; maar ik studeer nog verder voor een master': 'College',
 'International Baccalaureate (pre-universitair diploma, ongeveer gelijk aan ASO) ': 'Seconday education',
 '4de jaar kso': 'Primary education',
 'Hogeschool: lange type': 'College',
 'Specialisatiejaar hogeschool': 'College',
 'Hbo 5 verpleegkunde': 'College',
 'graduaat HBO5': 'College',
 'Graduaat (volwassenonderwijs)': 'College',
 'Bezig met universitaire opleiding, dus voorlopig alleen diploma van ASO secundair onderwijs': 'Seconday education',
 'Hoger beroepsonderwijs': 'College',
 'zorgkundige': 'Seconday education',
 'Hb05 verpleging ': 'College',
 'Hogeschool: Master': 'College',
 'Hbo-opleiding ': 'College',
 'Bachelor na bachelor': 'College',
 'HBO5': 'College',
 'deeltijds onderwijs': 'Secondary education',
 'MBO4': 'Seconday education',
 'bachelor na bachelor': 'College',
 'Hbo5': 'College',
 'Banaba': 'College',
 'Banaba Onco': 'College',
 'Manama': 'University',
 'Nog studerend: TSO middelbaar': 'Primary education',
 'Hbo': 'College',
 'Laatstejaars bachelor': 'Seconday education',
 'Hoger onderwijs kort type': 'College',
 'Technisch administratief': 'College', 
 'Hoger middelbaar kantoor/handel': "Secondary education",
 'Student op Universiteit Antwerpen (2de Bachelor momenteel) ': "Secondary education",
 'hogeschool: master': 'College',
 'HBO5 hoger beroepsonderwijs verpleegkunde ': 'College',
 'Gebrevetteerd verpleegkundige A2': 'College',
 'Hbo propedeuse ': "Secondary education",
 'MBO (middelbaar beroepsonderwijs = nederland niveau)': "Secondary education",
 'Momenteel bezig met Biomedische wetenschappen aan de universiteit Antwerpen, 2e bachelor': "Secondary education",
 'laatste jaar master momenteel': "University",
 'Hogeschool: bachelor - momenteel studeer ik voor mijn master (afstudeerjaar 2021)': 'College',
 'postgraduaat na master': "University",
 'Vwo diploma Nederland, bezig met hogeschool belgie': "Seconday education",
 'A2 verpleging': 'College',
 'Master na master': "University",
 'Graduaat': 'College',
 'Hbo5 verpleegkunde': 'College',
 'Hogeschool: lange type (master)': 'College',
 'verdergezet secundair ': "Secondary education",
 'Deeltijds kunstonderwijs': None,
 '4 jaar handel gestopt op 16 jaar': 'Primary education',
 'Mavo': 'Secondary education',
 'Volwassenenonderwijs ': 'College',
 'BanaBa en een postgraduaat opleiding UGent': 'College',
 'volwassenonderwijs syntra ': 'College',
 'HBOV verpleegkunde': 'College',
 'hogeschool beroepsopleiding': 'College',
 'Nog steeds student in een masteropleiding ': 'University',
 'Hogeschool: graduaat': 'College',
 'Master na Master ': 'University',
 '2 masters (universiteit)': 'University',
 'Hoger beroeps onderwijs; gegradueerde ': 'College',
 'Secundair na secundair (Se-n-Se)': 'Secunday education',
 'Bezig met bachelor opleiding vroedkunde': 'Secondary education',
 'BanaBa': 'College',
 'zit in mijn 1ste jaar bachelor aan de hogeschool ': 'Secondary education',
 'bezig ad hbo5 opleiding': 'Secondary education',
 'getuigschrift 6e middelbaar BSO': 'Secundary education',
 'Se-n-Se': 'Secundary education',
 'Ga nog naar school': None,
 'graduaat ': 'College',
 'Hogeschool ': 'College',
 'HBO verpleegkunde': 'College',
 'Havo': 'College',
 '2 de graad secundair tso': 'Primary education',
 'leercontract': None,
 'Secundair onderwijs door volwassenenonderwijs + certificaat webdesigner': 'Secundary education',
 'VWO': 'Secondary education',
 'BSO + 7e specialisatie=A2': 'Secondary education',
 'Diploma SLO opleiding ': 'College',
 'Ma na Ma': 'University',
 '7de jaar zorgkundige ': 'Secondary education',
 'Hbo5 ': 'College',
 'Mbo': 'Secondary education',
 'Volwassenonderwijs lerarendiploma ': 'College',
 'Graduaat ': 'College',
 'Hoger beroeps onderswijs': 'College',
 'Avondschool': None,
 'master na master': 'University',
 'Specialisatie 7e jaar na 6 jaar TSO': 'Secondary education',
 'Getuigschrift ': None,
 'Volwassenonderwijs ': None,
 'Hoger onderwijs van het korte type voor sociale promotie (hoktsp) Hoger onderwijs voor ': 'College',
 'HBO master': 'College',
 'Hoger beroeps onderwijs a2 verpleegkundige ': 'Seconday education',
 'Hoger onderwijs: Graduaat': 'College',
 'Hogeschoolcertificaat': 'College',
 'A2 verpleging ': 'Secondary education',
 'Hogeschool: bachelor & bachelor-na-bachelor': 'College',
 'MBO 4 Nederlands onderwijs': 'Secondary education',
 'HBO5 + SLO': 'College',
 'Volwassenonderwijs bedrijfsbeheer ': None,
 'MBO niv. 4': 'Secondary education',
 'Slo opleiding ': 'College',
 'graduaat': 'College',
 'Avondonderwijs': None,
 'postgraduaat': 'College',
 'Secundair diploma Amerika': 'Secondary education',
 'professionele master': 'College',
 'ManaMa': 'University',
 'Nederlandse opleiding MBO 4 juridisch medewerker ': 'College',
 'Master + banaba + postgraduaat': 'College',
 'Master-na-master (business school)': 'University',
 'Middelbare school kader Vmbo (Nederland)': 'Secondary education',
 'HBO graduaat verzekeringen + PCP statuut behaald via Febelfin (Overheid)': 'College',
 'Mbo 3': 'Secondary education',
 'Middelbaar beroeps onderwijs': 'Secondary education',
 'mbo': 'Secundary education',
 'Graduaat (HBO5)': 'College',
 'Hoger secundair, HBO5 verpleegkunde': 'College',
 'attest BUSO': 'Secondary education',
 'SNIT EN NAAD': 'Secondary education',
 'Bachelor,  korte type ': 'College',
 'Post-Graduaat': 'College',
 'GPB: lerarenopleiding ': 'College'
}

def c(row):
    main = row['Q1.Q18.[Wat is het hoogste diploma dat je behaalde? - Selected Choice]']
    other = row['Q1.Q18_9_TEXT.[Wat is het hoogste diploma dat je behaalde? - Andere - Tekst]']

    return map_main.get(main, map_other.get(other, None))

#edef

QC['General.Highest.Education'] = Qraw.apply(c, axis=1)

## Health

### BMI

In [150]:
#BMI

QC['Health.BMI'] = QC['General.Weight']/((QC['General.Height']/100)**2)

### General health

In [151]:
freq(Qraw['Q1.Q34.[Hoe beoordeel je je algemene\ngezondheidstoestand?]'])

{None: 1325,
 'Goed': 2811,
 'Redelijk': 638,
 'Zeer goed': 1171,
 'Slecht': 60,
 'Zeer slecht': 1,
 '': 1}

## Medication

In [190]:
possible_meds = {
"amox" : ["qmox", "alox"],
    "ibuprofen": ["ibbux", ...]
}

def flatten(lst, dropna=False):
    return [ v for l in lst for v in l if not (dropna & pd.isna(v)) ]
#edef

rel_cols = [ c for c in Qraw.columns if ('Q1.Q42' in c) and ('Naam' in c) ]

sorted(list(freq([ v.strip() for v in flatten(Qraw[rel_cols].values, True) if v != '']).items()), key=lambda x:x[0])

def _clean_meds(row, values):
    row_values = set([ v.strip() for v in row[rel_cols] if not pd.isna(v) ])

    return len(row_values & values) > 0
#edef

for med in possible_meds:
    QC["Health.medication.%s" % med] = Qraw[rel_cols].apply(lambda row:_clean_meds(row, set(possible_meds[med])))
#efor

[('(iets met een az)', 1),
 ('-', 1),
 ('/', 3),
 ('500mg gambaran', 1),
 ('?', 8),
 ('? (algemeen antibioticum)', 1),
 ('? Antibiotica?', 1),
 ('? breed spectrum', 1),
 ('A.D.P oregano-olie', 1),
 ('AMOXICLAV', 1),
 ('ANTIBIOTICA', 1),
 ('AO complex', 1),
 ('Aaceefemine ovules', 1),
 ('Abasaglar', 1),
 ('Acciclovir', 1),
 ('Aceclofenac', 3),
 ('Aceclovir', 1),
 ('Acerola 1000', 1),
 ('Acetilsysteine', 1),
 ('Acetylcysteïne', 1),
 ('Acetylsalicylzuur', 1),
 ('Aciclovir', 6),
 ('Aciclovir 800mg', 1),
 ('Aciclovir Sandoz', 4),
 ('Aciclovir Sandoz 800 mg', 1),
 ('Aclovir', 1),
 ('Aconitum Napellus', 1),
 ('Adalat', 1),
 ('Adalat Oros', 1),
 ('Advantan', 1),
 ('Advantan crème', 1),
 ('Aecefemine 2mg', 1),
 ('Aerinaze', 1),
 ('Aerius', 27),
 ('Aerius 5mg', 1),
 ('Aimovig', 2),
 ('Airflusa', 1),
 ('Airflusal', 1),
 ('Aldactone', 2),
 ('Alendronate', 1),
 ('Algocod', 1),
 ('Algotra', 3),
 ('Allegratab', 1),
 ('Alleophta unidosis', 1),
 ('Allergie pillen', 1),
 ('Almogran', 1),
 ('Almotriptan'

In [165]:
rel_cols = [ c for c in Qraw.columns if 'Q1.Q37_9' in c]
rel_cols

['Q1.Q37_9.[Heb je één van onderstaande aandoeningen? (Graag de aandoening\nvermelden) (meerdere antwoorden mogelijk) - Selected Choice - Voortplantingsstelsel, vb. endometriose]',
 'Q1.Q37_9_TEXT.[Heb je één van onderstaande aandoeningen? (Graag de aandoening\nvermelden) (meerdere antwoorden mogelijk) - Voortplantingsstelsel, vb. endometriose - Tekst]']

## Sexual

### Intercourse 3 months

In [160]:

freq(Qraw['Q1.Q47.[Heb je de afgelopen drie maanden één of meerdere seksuele\npartner(s) gehad? - Selected Choice]'])

Sexual.Partners.3months

{None: 1325,
 'Ja, één vaste partner': 3566,
 'Nee': 744,
 'Ja, meerdere vaste partners: hoeveel?': 131,
 'Ja, wisselende partners: hoeveel?': 240,
 '': 1}

In [161]:
freq(Qraw['Q2.Q26_15_TEXT.[Waarom heb je de afgelopen maand geen anticonceptie methode gebruikt? (Meerdere antwoorden mogelijk) - Andere: - tekst]'])

{None: 2753,
 '': 3096,
 'menopauze': 12,
 'ik ben in menopauze': 1,
 'Menopauze': 13,
 'Problemen met spiraal, dus heb deze laten weghalen': 1,
 'Ik ben zwanger. ': 1,
 'MENOPAUZE': 1,
 'ik ben zwanger': 6,
 'recent gescheiden, ex-partner was gesteriliseerd vandaar al jaren geen anticonceptie meer gebruikt': 1,
 "Neem al heel lang anticonceptie om maandelijkse bloedingen 'onder controle' te houden. Ik vond dat het tijd was voor een break. Met als gevolg dat ik nu zo goed als constant bloedverlies heb (+/- 2maanden met ups and downs)": 1,
 'te oud': 1,
 'Gemenoposeerd': 1,
 'Ik heb geen partner': 2,
 'Ik wou geen hormonen meer': 1,
 'ik ben zwanger geworden': 1,
 'Ben voorbij de menopauze': 1,
 'Ik heb geen relatie': 1,
 'Ik ben in de menopauze ': 1,
 'wij hebben seks zonder penetratie': 1,
 'Eierstokken operatief verwijderd (cyste)': 1,
 'Vanwege factor v van Leiden is het niet aangewezen': 1,
 'te veel negatieve invloed, alles geprobeerd': 1,
 'ik ben 54': 1,
 'Te vaak tussenbloeding

In [163]:
freq(Qraw['Q2.Q28_7_TEXT.[Wanneer was de eerste dag van je laatste menstruatie? - De eerste dag van mijn laatste menstruatie was (dd/mm/jjjj): - tekst]'])

{None: 2753,
 '': 943,
 '05/08/2020': 25,
 '23/02/2020': 1,
 '02/08/2020': 22,
 '25/07/2020': 22,
 '14/02/2020': 1,
 '11/07/2020': 41,
 '15/06/2020': 23,
 '18/07/2020': 45,
 '31/07/2020': 14,
 '22/07/2020': 37,
 '29/07/2020': 14,
 '13/08/2020': 8,
 '15/08/2020': 10,
 '10/07/2020': 58,
 '01/08/2020': 18,
 '12/06/2020': 8,
 '19/07/2020': 36,
 '14/08/2020': 9,
 '30/07/2020': 28,
 '24/07/2020': 35,
 '21/08/2020': 4,
 '08/08/2020': 10,
 '16/07/2020': 45,
 '12/08/2020': 12,
 '04/08/2020': 23,
 '06/08/2020': 15,
 '14/07/2020': 58,
 '28/08/2020': 3,
 '07/08/2020': 11,
 '23/07/2020': 37,
 '15/07/2020': 44,
 '27/07/2020': 27,
 '05/06/2020': 6,
 '17/07/2020': 47,
 '09/07/2020': 51,
 '15-07-2020': 2,
 '28/07/2020': 10,
 '30/06/2020': 46,
 '26/08/2020': 3,
 '26/07/2020': 30,
 '21/06/2020': 23,
 '09/08/2020': 10,
 '01/07/2020': 56,
 '19/08/2020': 5,
 '08/07/2020': 75,
 '25/05/2020': 7,
 '27/02/2020': 1,
 '03/08/2020': 12,
 '20/08/2020': 3,
 '14/06/2020': 17,
 '08/06/2020': 8,
 '19/06/2020': 19,
 '21

## Reproductive

### Number of biological children

In [100]:
freq(Qraw['Q1.Q6_1_TEXT.[Heb je kinderen? (Meerdere antwoorden mogelijk) - Biologische kinderen: hoeveel? - Tekst]'])

{None: 1325, '': 2918, 2: 828, 3: 275, 1: 597, 4: 52, 6: 2, 5: 8, 7: 1, 8: 1}

In [102]:


def clean_nchildbio(val):
    if pd.isna(val):
        return None
    elif val1 == '':
        return 0
    elif str(val).isnumeric():
        return int(val)
    else:
        return None
    #fi
#edef

QC['Reproductive.Children.n.biological'] = Qraw["Q1.Q6_1_TEXT.[Heb je kinderen? (Meerdere antwoorden mogelijk) - Biologische kinderen: hoeveel? - Tekst]"].apply(clean_nchildbio)

### Number of pregnancies

In [19]:
def clean_npreg(val):
    if pd.isna(val):
        return None
    elif str(val).isnumeric():
        return int(val)
    else:
        return None
    #fi
#edef

QC['Reproductive.Pregnancy.n'] = Qraw["Q1.Q50.[Hoeveel keer was je al zwanger?]"].apply(clean_npreg)

### Early menopause

In [139]:

def clean_earlymenopause(val):
    if pd.isna(val):
        return None
    else:
        return 'menopauze' in val
    #fi
#edef

QC['Reproductive.PeriMenopause.Early'] = Qraw['Q1.Q37_9_TEXT.[Heb je één van onderstaande aandoeningen? (Graag de aandoening\nvermelden) (meerdere antwoorden mogelijk) - Voortplantingsstelsel, vb. endometriose - Tekst]'].apply(clean_earlymenopause)

## Sterility

In [167]:
## rel_cols = [ c for c in Qraw.columns if 'Q1.Q37_9' in c]
rel_cols

['Q1.Q37_9.[Heb je één van onderstaande aandoeningen? (Graag de aandoening\nvermelden) (meerdere antwoorden mogelijk) - Selected Choice - Voortplantingsstelsel, vb. endometriose]',
 'Q1.Q37_9_TEXT.[Heb je één van onderstaande aandoeningen? (Graag de aandoening\nvermelden) (meerdere antwoorden mogelijk) - Voortplantingsstelsel, vb. endometriose - Tekst]']

### Current alcohol yes/no

In [126]:

def current_alcohol(val):
    if pd.isna(val):
        return None
    else:
        return val != 'Ik drink geen alcohol'
    #fi
#edef

QC['Lifestyle.Drugs.Current_alcohol'] = Qraw['Q1.Q30_4.[Als je alcohol drinkt, wat drink je dan meestal? (meerdere antwoorden mogelijk) - Selected Choice - Ik drink geen alcohol]'].apply(current_alcohol)

### Stress

In [201]:
freq(Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'])



{None: 1325,
 'Vaak': 1170,
 'Soms': 2137,
 'Zelden': 767,
 'Voortdurend': 133,
 'Meestal': 399,
 'Nooit': 75,
 '': 1}

In [208]:

#Binary variable
def clean_stress(val):
    if pd.isna(val) or val == '':
        return None
    else :
        return val in ['Vaak', 'Meestal', 'Voortdurend']
    #fi
#edef


QC['Lifestyle.Mood.Stress.4weeks'] = Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'].apply(clean_stress)

#Dummy variables

##Voortdurend
def clean_stress_voortdurend(val):
    if pd.isna(val) or val == '':
        return None
    else :
        return val == 'Voortdurend'
    #fi
#edef
QC['Lifestyle.Mood.Stress.4weeks.Voortdurend'] = Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'].apply(clean_stress_voortdurend)

##Meestal
def clean_stress_meestal(val):
    if pd.isna(val) or val == '':
        return None
    else :
        return val == 'Meesatl'
    #fi
#edef
QC['Lifestyle.Mood.Stress.4weeks.Meestal'] = Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'].apply(clean_stress_meestal)

##Vaak
def clean_stress_vaak(val):
    if pd.isna(val) or val == '':
        return None
    else :
        return val == 'Vaak'
    #fi
#edef
QC['Lifestyle.Mood.Stress.4weeks.Vaak'] = Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'].apply(clean_stress_vaak)

##Soms
def clean_stress_soms(val):
    if pd.isna(val) or val == '':
        return None
    else :
        return val == 'Soms'
    #fi
#edef
QC['Lifestyle.Mood.Stress.4weeks.Soms'] = Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'].apply(clean_stress_soms)

##Zelden
def clean_stress_zelden(val):
    if pd.isna(val) or val == '':
        return None
    else :
        return val == 'Zelden'
    #fi
#edef
QC['Lifestyle.Mood.Stress.4weeks.Zelden'] = Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'].apply(clean_stress_zelden)

##Nooit
def clean_stress_nooit(val):
    if pd.isna(val) or val == '':
        return None
    else :
        return val == 'Nooit'
    #fi
#edef
QC['Lifestyle.Mood.Stress.4weeks.Nooit'] = Qraw['Q1.Q46_6.[Hoe voelde je je de afgelopen vier weken? - Voelde je je gestresseerd?]'].apply(clean_stress_nooit)


In [209]:
QC

,IsalaID,Reproductive.Cycle.Phase.Luteal,Reproductive.Cycle.Phase.Ovulation,Reproductive.Cycle.Phase.Follicular,Reproductive.PeriMenopause,Lifestyle.Partner.IsMan,Lifestyle.Partner.IsWoman,Reproductive.Pregnancy.Miscarriage,Reproductive.Pregnancy.PretermBirth,Reproductive.Pregnancy.Abortion,...,Reproductive.PeriMenopause.Early,Health.BMI,Lifestyle.Mood.Stress.4weeks,General.Highest.Education,Lifestyle.Mood.Stress.4weeks.Voortdurend,Lifestyle.Mood.Stress.4weeks.Meestal,Lifestyle.Mood.Stress.4weeks.Vaak,Lifestyle.Mood.Stress.4weeks.Soms,Lifestyle.Mood.Stress.4weeks.Zelden,Lifestyle.Mood.Stress.4weeks.Nooit
idx,,,,,,,,,,,,,,,,,,,,,
ISALA00006,ISALA00006,None,None,None,False,None,None,False,False,False,...,None,NaN,None,None,None,None,None,None,None,None
ISALA00038,ISALA00038,None,None,None,False,None,None,False,False,False,...,None,NaN,None,None,None,None,None,None,None,None
ISALA00039,ISALA00039,None,None,None,False,True,False,False,False,False,...,False,20.199470,True,PhD,False,False,True,False,False,False
ISALA00040,ISALA00040,None,None,None,False,True,False,False,False,False,...,False,21.258503,True,College,False,False,True,False,False,False
ISALA00041,ISALA00041,None,None,None,False,None,None,False,False,False,...,None,NaN,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISALA06046,ISALA06046,None,None,None,False,True,False,False,False,False,...,False,24.725183,False,University,False,False,False,True,False,False
ISALA06047,ISALA06047,None,None,None,False,None,None,True,False,False,...,False,16.836011,False,University,False,False,False,True,False,False
ISALA06048,ISALA06048,None,None,None,False,None,None,False,False,False,...,None,NaN,None,None,None,None,None,None,None,None


In [29]:
list(Qraw)

['Q0.Status',
 'Q0.Deelnemersnummer',
 'Q0.Leeftijd',
 'Q0.Woonde in België voor de eerste 3 levensjaren',
 'Q0.Zwanger',
 'Q0.Anticonceptiemethode',
 'Q0.Postcode',
 'Q1.StartDate.[Start Date]',
 'Q1.EndDate.[End Date]',
 'Q1.Status.[Response Type]',
 'Q1.Progress',
 'Q1.Duration (in seconds)',
 'Q1.Finished',
 'Q1.RecordedDate.[Recorded Date]',
 'Q1.ResponseId.[Response ID]',
 'Q1.ExternalReference.[External Data Reference]',
 "Q1.IC4.[Ik heb het informatie- en toestemmingsformulier\ngelezen. Ik begrijp de voordelen en het belang van deze studie, maar ook de eventuele\nrisico's. Ik begrijp dat ik binnen het onderzoek een uitgebreide vragenlijst\nkrijg om te bepalen welke factoren een effect zouden kunnen hebben op het\nmicrobioom. Ik neem vrijwillig deel aan deze studie. Vandaag ([CurrentDate-d%2Fm%2FY])]",
 'Q1.IC5.[Je kan alleen deelnemen aan Isala als je akkoord gaat met het toestemmingsformulier. Indien je hierover vragen hebt, kan je contact met ons opnemen via isala@uantwerpen.

# Export cleaned questionnaire

In [114]:
with RDM('../data/cleaned/IsalaFlow1.CleanedQuestionnaires.xlsx', 'w', nouser=True) as ofd:
    QC.to_excel(ofd)
#ewith

with RDM('../data/cleaned/IsalaFlow1.CleanedQuestionnaires.xlsx', 'w', nouser=True) as ofd:
    QC.to_pickle(ofd)
#ewith

NameError: name 'RDM' is not defined